# Image Table Scraper
This notebook loads `sw_images/engagement.png`, uses EasyOCR to extract text, identifies the table starting with "Metric", and creates a DataFrame.

In [1]:
# Install dependencies if not present
# !pip install easyocr pandas opencv-python-headless

import easyocr
import pandas as pd
import numpy as np
import warnings
import re
import glob
import os
warnings.filterwarnings("ignore")

In [2]:
image_path = 'sw_images/engagement.png'
print(f"Target Image: {image_path}")

Target Image: sw_images/engagement.png


In [3]:
# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'])

# Read text from image
# output format: [[bounding_box], text, confidence]
results = reader.readtext(image_path)

# Filter for useful low-confidence text if needed, but usually default is fine
print(f"Extracted {len(results)} text elements.")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Extracted 50 text elements.


In [4]:
# Logic to reconstruct the table

# 1. Convert results to a list of dicts for easier handling
text_data = []
for (bbox, text, prob) in results:
    # bbox points: [top_left, top_right, bottom_right, bottom_left]
    # Get center y for row grouping
    tl, tr, br, bl = bbox
    y_center = (tl[1] + bl[1]) / 2
    x_center = (tl[0] + tr[0]) / 2
    text_data.append({'text': text, 'y': y_center, 'x': x_center, 'bbox': bbox})

# 2. Group by Rows (grouping text elements that are on the same line with some tolerance)
df_text = pd.DataFrame(text_data)
df_text = df_text.sort_values(by='y')

rows = []
current_row = []
last_y = -1
y_tolerance = 15  # Adjust based on image resolution

for index, row in df_text.iterrows():
    if last_y == -1 or abs(row['y'] - last_y) < y_tolerance:
        current_row.append(row)
        # update last_y to average or keep it? moving average might be better for slight skews
        last_y = row['y']
    else:
        # Sort the completed row by X to get columns
        current_row.sort(key=lambda x: x['x'])
        rows.append(current_row)
        
        # Start new row
        current_row = [row]
        last_y = row['y']

# Append the last row
if current_row:
    current_row.sort(key=lambda x: x['x'])
    rows.append(current_row)

# 3. Find the Header Row (containing "Metric")
header_row_index = -1
for i, row in enumerate(rows):
    texts = [item['text'] for item in row]
    if "Metric" in texts:
        header_row_index = i
        break

# 4. Extract Header + 7 Data Rows
final_rows_data = []

if header_row_index != -1:
    # Extract Header
    # Sometimes OCR splits "mobitel.lk" or "Metric", so we take the list of text
    header_texts = [item['text'] for item in rows[header_row_index]]
    print("Header Found:", header_texts)
    
    # Extract next 7 rows
    # Note: Depending on spacing, there might be empty rows or noise.
    # We iterate and pick the next 7 valid-looking rows (or just the next 7 in list if clean)
    
    data_start_idx = header_row_index + 1
    data_rows_captured = 0
    
    # We want to form a table. For simplicity, we'll just grab the text lists.
    # A more robust way aligns them to header columns by X-coordinate, 
    # but for now let's assume they are detected in order.
    
    extracted_table = []
    # Header
    extracted_table.append(header_texts)
    
    for i in range(data_start_idx, len(rows)):
        if data_rows_captured >= 7:
            break
        
        row_texts = [item['text'] for item in rows[i]]
        
        # FIX: Merge 'Visits' and 'Unique visitors' if split
        if len(row_texts) > 1 and "Visits" in row_texts[0] and "Unique" in row_texts[1]:
            row_texts[0] = row_texts[0] + " / " + row_texts[1]
            row_texts.pop(1)
            
        # Simple heuristic to skip noise: row must have at least 2 items
        if len(row_texts) >= 2:
            extracted_table.append(row_texts)
            data_rows_captured += 1
    
    # Create DataFrame
    # Note: Row lengths might mismatch if OCR missed some values (e.g. "-" or small text)
    # We'll normalize length to the header length
    header = extracted_table[0]
    data = extracted_table[1:]
    
    # Ensure all data rows match header length by padding or trimming
    # (In a real scenario, we'd map by X-coordinate to handle missing cells)
    normalized_data = []
    for r in data:
        if len(r) < len(header):
            r = r + [None] * (len(header) - len(r))
        elif len(r) > len(header):
             # This is tricky; maybe OCR merged header columns or split data columns.
             # For now, slice it.
            r = r[:len(header)]
        normalized_data.append(r)
        
    df = pd.DataFrame(normalized_data, columns=header)

    # Transpose DataFrame as requested
    if 'Metric' in df.columns:
        df_engagement = df.set_index('Metric').T
    else:
        # Fallback if OCR messed up the column name 'Metric'
        # Assume first column is metric
        df_engagement = df.set_index(df.columns[0]).T
        
    # Data Cleaning Logic
    def clean_value(x):
        if not isinstance(x, str):
            return x
        x = x.strip()
        
        # Handle explicitly known non-numeric placeholders
        if x.upper() in ['N/A', 'NIA', 'NAN', 'NONE', '', '-']:
            return 0
        
        # Handle M (Millions)
        if 'M' in x:
            try:
                return float(x.replace('M', '').replace(',', '')) * 1_000_000
            except:
                return 0
        # Handle % (Percentage)
        if '%' in x:
            try:
                return float(x.replace('%', '').replace(',', '')) * 0.01
            except:
                return 0
        # Try to convert string numbers to float/int if possible (handle commas)
        try:
            val_cleaned = x.replace(',', '')
            if '.' in val_cleaned:
                return float(val_cleaned)
            else:
                return int(val_cleaned)
        except:
            return 0

    df_engagement = df_engagement.applymap(clean_value)
        
else:
    print("Could not find a row containing 'Metric'.")
    df_engagement = pd.DataFrame()

Header Found: ['Metric', 'mobitel Ik', 'dialog Ik', 'hutch Ik', 'airtel Ik', 'slt Ik']


In [5]:
# Display Transposed DataFrame
print("Reference File:", image_path)
df_engagement

Reference File: sw_images/engagement.png


Metric,Monthly visits,Monthly unique visitors,Visits / Unique visitors,Visit duration,Pages per visit,Bounce rate,Page Views
mobitel Ik,1281000.0,399619,3.21,0,3.61,0.3797,4628000.0
dialog Ik,2689000.0,971059,2.77,0,2.73,0.5290,7332000.0
hutch Ik,506387.0,168630,3.00,0,2.60,0.5574,1314000.0
airtel Ik,623606.0,144970,4.30,0,2.65,0.5020,1652000.0
slt Ik,1354000.0,282834,4.79,0,4.42,0.3097,5990000.0


## Social Network Data Extraction
Processing `sw_images/social_network.png` to extract the defined table.

In [6]:
# Social Network Extraction
sn_image_path = 'sw_images/social_network.png'
print(f"Target Image: {sn_image_path}")

# Read text from image
# Using regex to fix common issues in post-processing
# Adjusting mag_ratio to catch small text
sn_results = reader.readtext(sn_image_path, mag_ratio=1.5)

# 1. Convert to list of dicts
sn_text_data = []
for (bbox, text, prob) in sn_results:
    tl, tr, br, bl = bbox
    y_center = (tl[1] + bl[1]) / 2
    x_center = (tl[0] + tr[0]) / 2
    
    # CLEANUP: 
    # 1. Remove '<' (e.g. '< 0.01%')
    text = text.replace('<', '').strip()
    
    # 2. Fix '9' at end instead of '%'
    # Pattern: Digit(s) + '9' -> Digit(s) + '%'
    if re.match(r'^[\d\.]*9$', text) and len(text) > 1:
        text = text[:-1] + '%'
    
    sn_text_data.append({'text': text, 'y': y_center, 'x': x_center})

# 2. Group by Rows
df_sn_text = pd.DataFrame(sn_text_data)
df_sn_text = df_sn_text.sort_values(by='y')

sn_rows = []
sn_current_row = []
sn_last_y = -1

for index, row in df_sn_text.iterrows():
    if sn_last_y == -1 or abs(row['y'] - sn_last_y) < y_tolerance:
        sn_current_row.append(row)
        sn_last_y = row['y']
    else:
        sn_current_row.sort(key=lambda x: x['x'])
        sn_rows.append(sn_current_row)
        sn_current_row = [row]
        sn_last_y = row['y']

if sn_current_row:
    sn_current_row.sort(key=lambda x: x['x'])
    sn_rows.append(sn_current_row)

# 3. Find Header (Looking for "Network")
sn_header_row_index = -1
for i, row in enumerate(sn_rows):
    texts = [item['text'] for item in row]
    if any("Network" in t for t in texts):
        sn_header_row_index = i
        break

# 4. Extract Data with X-Coordinate Alignment
if sn_header_row_index != -1:
    # Get Headers with their X Coordinates
    header_items = sn_rows[sn_header_row_index]
    sn_headers = [{'text': item['text'], 'x': item['x']} for item in header_items]
    print("Social Headers:", [h['text'] for h in sn_headers])
    
    sn_extracted_data = []
    
    count = 0
    for i in range(sn_header_row_index + 1, len(sn_rows)):
        if count >= 7: break
        
        row_items = sn_rows[i]
        if len(row_items) < 1: continue
        
        # Initialize row with Nones
        mapped_row = {h['text']: None for h in sn_headers}
        
        # Map each item to the nearest header
        for item in row_items:
            closest_header = min(sn_headers, key=lambda h: abs(h['x'] - item['x']))
            
            # Check if existing value is there (collision)
            # If so, maybe concatenate or choose better?
            if mapped_row[closest_header['text']] is not None:
                 # Append or overwrite? For now append to debug
                 mapped_row[closest_header['text']] += " " + item['text']
            else:
                mapped_row[closest_header['text']] = item['text']
        
        sn_extracted_data.append(mapped_row)
        count += 1
            
    df_sn = pd.DataFrame(sn_extracted_data)
    
    # Replace None with "0%"
    df_sn = df_sn.fillna("0%")
    
    print("Raw Social DataFrame (Corrected Alignment):")
    display(df_sn)
    
    # Transpose
    network_col = next((c for c in df_sn.columns if "Network" in c), None)
    if network_col:
        df_social = df_sn.set_index(network_col).T
    else:
        df_social = df_sn.set_index(df_sn.columns[0]).T
        
    print("Transformed Social DataFrame:")
    display(df_social)
else:
    print("Could not find 'Network' header row.")
    df_social = pd.DataFrame()

Target Image: sw_images/social_network.png


Social Headers: ['Networks', 'mobitel Ik', 'dialog Ik', 'hutchIk', 'airtel Ik', 'slt Ik']
Raw Social DataFrame (Corrected Alignment):


,Networks,mobitel Ik,dialog Ik,hutchIk,airtel Ik,slt Ik
0,Youtube,13.08%,18.79%,20.10%,0%,67.25%
1,Facebook,0%,17.15%,61.78%,35.76%,22.14%
2,Linkedin,14.25%,40.64%,0%,0%,0%
3,WhatsApp Webapp,14.67%,20.79%,0%,0%,6.79%
4,Social,26.00%,0.01%,18.12%,64.24%,0.01%
5,Academia,31.99%,0%,0%,0%,0%
6,Others,0%,2.63%,0%,0%,3.82%


Transformed Social DataFrame:


Networks,Youtube,Facebook,Linkedin,WhatsApp Webapp,Social,Academia,Others
mobitel Ik,13.08%,0%,14.25%,14.67%,26.00%,31.99%,0%
dialog Ik,18.79%,17.15%,40.64%,20.79%,0.01%,0%,2.63%
hutchIk,20.10%,61.78%,0%,0%,18.12%,0%,0%
airtel Ik,0%,35.76%,0%,0%,64.24%,0%,0%
slt Ik,67.25%,22.14%,0%,6.79%,0.01%,0%,3.82%


## Channel Traffic Extraction
Looping through `channel_traffic-*.png` to extract channel-specific data and merge into a single summary table.

In [7]:
# Channel Traffic Extraction Logic

channel_files = glob.glob('sw_images/channel_traffic-*.png')
print(f"Found {len(channel_files)} channel files.")

# Placeholder for the merged DataFrame
df_channels = pd.DataFrame()

for fpath in channel_files:
    # Extract suffix from filename (e.g. 'direct' from 'channel_traffic-direct.png')
    base_name = os.path.basename(fpath)
    # format is 'channel_traffic-<suffix>.png'
    # split by '-' then remove extension
    # assumption: filename format is strict
    try:
        suffix = base_name.split('channel_traffic-')[1].replace('.png', '')
    except IndexError:
        print(f"Skipping file with unexpected format: {base_name}")
        continue

    print(f"Processing: {base_name} -> Column: {suffix}")
    
    # OCR
    # Using mag_ratio=1.5 for consistency, though default might suffice for larger text
    results = reader.readtext(fpath, mag_ratio=1.5)
    
    # Standard Row Grouping Logic (reused)
    c_text_data = []
    for (bbox, text, prob) in results:
        tl, tr, br, bl = bbox
        y_center = (tl[1] + bl[1]) / 2
        x_center = (tl[0] + tr[0]) / 2
        
        # Clean values immediately if simple
        text = text.replace('<', '').strip()
        # Fix common digit-percent error
        if re.match(r'^[\d\.]*9$', text) and len(text) > 1:
            text = text[:-1] + '%'
            
        c_text_data.append({'text': text, 'y': y_center, 'x': x_center})
        
    df_c = pd.DataFrame(c_text_data)
    if df_c.empty:
        print(f"No text found in {base_name}")
        continue
        
    df_c = df_c.sort_values(by='y')
    
    c_rows = []
    c_current = []
    last_y = -1
    # Use same tolerance
    y_tolerance = 15
    
    for index, row in df_c.iterrows():
        if last_y == -1 or abs(row['y'] - last_y) < y_tolerance:
            c_current.append(row)
            last_y = row['y']
        else:
            c_current.sort(key=lambda x: x['x'])
            c_rows.append(c_current)
            c_current = [row]
            last_y = row['y']
    if c_current:
         c_current.sort(key=lambda x: x['x'])
         c_rows.append(c_current)

    # Construct DataFrame for this file
    # Expectation: Each row has 2 items: [Website Name, Value]
    # or loop until we find rows with >= 2 items or looks like a table
    extracted_data = []
    
    for row in c_rows:
        texts = [item['text'] for item in row]
        # Heuristic: skip headers like 'Website', 'Direct', etc. if they appear
        # We assume the actual data rows contain the website names we know (mobitel, dialog etc) 
        # OR we just grab everything with 2 columns and filter later.
        
        # Let's filter for rows with exactly 2 items, as presumably these are [site, value]
        # If there are more, OCR might have split a name.
        if len(texts) >= 2:
            # Assuming First is Website, Last is Value (middle might be noise or split name)
            # If split name: "mobitel", "Ik" -> join them?
            # Simple join of all except last is safer for name
            website = " ".join(texts[:-1])
            value = texts[-1]
            
            # NORMALIZATION START
            # 1. Replace '.' with ' ' (e.g. 'airtel.Ik' -> 'airtel Ik')
            website = website.replace('.', ' ')
            # 2. Fix missing space before 'Ik' (e.g. 'hutchIk' -> 'hutch Ik')
            if 'Ik' in website and ' Ik' not in website:
                website = website.replace('Ik', ' Ik')
            # 3. Strip extra whitespace
            website = website.strip()
            # NORMALIZATION END
            
            extracted_data.append({'Website': website, suffix: value})
            
    # Create temp DF
    df_temp = pd.DataFrame(extracted_data)
    
    # Clean Value Column immediately
    if not df_temp.empty:
        df_temp[suffix] = df_temp[suffix].apply(clean_value)
    
    # Merge
    if df_channels.empty:
        df_channels = df_temp
    else:
        # Merge on Website
        # Note: Website names must match EXACTLY relative to previous files for this to work perfectly.
        # If OCR varies ('mobitel Ik' vs 'mobitel lk'), we get dupes / NaNs.
        # For this exercise, we assume consistent OCR or accept slight fragmentation.
        df_channels = pd.merge(df_channels, df_temp, on='Website', how='outer')

print("Merged Channel Traffic DataFrame:")
display(df_channels)

Found 9 channel files.
Processing: channel_traffic-direct.png -> Column: direct


Processing: channel_traffic-display.png -> Column: display


Processing: channel_traffic-email.png -> Column: email


Processing: channel_traffic-gen_ai.png -> Column: gen_ai


Processing: channel_traffic-referrals.png -> Column: referrals


...

Processing: channel_traffic-search_paid.png -> Column: search_paid


Processing: channel_traffic-social_organic.png -> Column: social_organic


Processing: channel_traffic-social_paid.png -> Column: social_paid


Merged Channel Traffic DataFrame:


,Website,direct,display,email,gen_ai,referrals,search_organic,search_paid,social_organic,social_paid
0,airtel Ik,319637.0,81561,1467,0,100556,1315000.0,0,10127,42328
1,dialog Ik,3653000.0,177850,38060,35330,288045,3487000.0,173130,164103,49952
2,hutch Ik,696452.0,10020,6055,2045,55028,707877.0,0,15504,26181
3,mobitel Ik,1659000.0,28404,1538,13981,263114,1766000.0,84979,22774,4601
4,slt Ik,3180000.0,8196,7512,1908,138158,686412.0,8890,24524,7394
